## Langevin dynamics: introduction with simulations

Topics:
- Perform simple simulations of Langevin dynamics
- Verify the expected equilibrium properties
- Probe the dynamics with correlation functions

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import tidynamics

### A super mega crash course on Python

This introduction uses Python.

Let's go!

In [ ]:
# Variables

a = 1
print(a, type(a))
a = '1'
print(a, type(a))
a = 1.0
print(a, type(a))


In [ ]:
# Loops

for a in [1, '1', 1.0]:
    print(a, type(a))

In [ ]:
# Functions

def my_function(x, y):
    return x+y

print(my_function(1, 1))
print(my_function('one ', 'two'))
print(my_function(1.0, 2.0))
print(my_function(1.0, 'two'))

## Arrays in Python

In Python, the most convenient data type for numerical data is the NumPy array.

In [ ]:
my_array = np.ones(5)
print(my_array)
my_array[0] = 10
print(my_array)
print(2*my_array)
print(np.sin(my_array))

## The Langevin equation

The Langevin equation for the velocity (Ornstein-Uhlenbeck process)

$$\dot v = - \gamma v + \sqrt{2 \gamma T} \xi$$

where $\gamma$ is the friction coefficient, $T$ is the temperature ($k_B=1$) and
$\xi$ is gaussian white noise.

Questions:
1. How to represent the noise numerically?
2. How to do a simulation of this equation?

### White noise

Noise is generated from a "pseudo random number generator" (RNG or PRNG).

In [ ]:
sample_data = np.random.normal(size=100)

In [ ]:
plt.figure()
count, bins, patches = plt.hist(sample_data, bins=32, normed=True)
plt.plot(bins, np.exp(-bins**2/2)/np.sqrt(2*np.pi));

## What about the correlations?

A definining feature of the noise $\xi$ is its autocorrelation:

$$\langle \xi(t_1) \xi(t_2) \rangle = 2 T \gamma \delta(t_1-t_2)$$

**Exercise:** Compute the autocorrelation of the noise.

In discrete time, use

$$\langle \xi(t_1) \xi(t_2) \rangle = \frac{1}{\# \mathrm{ samples}} \sum_{i, j} \xi(i) \xi(j)$$

where the sum is taken over the i and j that obey $i-j = t_1-t_2$

In [ ]:
N_data_points = 1000
noise = np.random.normal(size=N_data_points)

autocorrelation = np.zeros(2*N_data_points - 1)
autocorrelation_count = np.zeros(2*N_data_points - 1)
center_point = N_data_points

for i in range(N_data_points):
    for j in range(N_data_points):
        pass        

autocorrelation /= autocorrelation_count

In [ ]:
plt.figure()
plt.plot(autocorrelation);

In [ ]:
x_axis = center_point + np.arange(len(noise)) - 1
plt.plot(x_axis, tidynamics.acf(noise))

## Solutions to the Langevin equation

$$v(t+dt) = v(t) + \int_t^{t+dt} dt' ~ \left[ -\gamma v(t') + \sqrt{2\gamma T} \xi(t')\right]$$

The Euler-Maruyama algorithm is the simplest stochastic integrator:

$$v(t+dt) \approx v(t) - \gamma v(t) dt + \sqrt{2\gamma T dt} \Gamma$$

For research projects, do a bit of research to find a better algorithm!

The Euler-Maruyama algorithm requires small timesteps.

**Exercise:** apply iteratively the euler step to collect a time series for the velocity.

**Exercise:** plot the equilibrium velocity distribution.

In [ ]:
def euler_step(v, gamma, T, dt):
    return v - gamma*v*dt + np.sqrt(2*gamma*T*dt)*np.random.normal()


In [ ]:
data = [] # Create an empty list

g = 0.1
dt = 0.01
T = 2

v = 0 # Initial condition

for i in range(1000):
    pass

for i in range(10000):
    for j in range(10):
        pass
    data.append(v) # Sample every 10 dt
data = np.array(data)

In [ ]:
plt.figure()

plt.plot(data)
plt.xlabel('Time') ; plt.ylabel('velocity')

In [ ]:
plt.figure()
count, bins, patches = plt.hist(data, bins=32, normed=True)

plt.plot(bins, bins) # Here, replace "y-axis" by the equilibrium distribution

plt.xlabel('velocity') ; plt.title('Histogram of velocity')


## Correlation with the force

Other data than the velocity autocorrelation can be useful.
The force - velocity correlation function shows, for instance, causality.

In [ ]:
# Define the Euler while also returning the value of the force

def euler_step_with_force(v, gamma, T, dt):
    force = np.sqrt(2*gamma*T*dt)*np.random.normal()
    return v - gamma*v*dt+ force, force/dt


In [ ]:
v_data = []
f_data = []

g = 0.1
dt = 0.01
T = 2

v = 0 # Initial condition

for i in range(1000):
    v = euler_step(v, g, T, dt)

for i in range(80000):
    for j in range(5):
        v, force = euler_step_with_force(v, g, T, dt)
    v_data.append(v)
    f_data.append(force)
v_data = np.array(v_data)
f_data = np.array(f_data)

In [ ]:
plt.figure()
tr = (-len(v_data) + np.arange(2*len(v_data)-1))*5*dt
plt.plot(tr, tidynamics.core.correlation_1d(f_data, v_data))
#plt.xlim(-10, 20)
plt.ylim(-1, 2)
plt.title(r'$\langle f(0) v(\tau) \rangle$')
plt.xlabel(r'$\tau$')

## Langevin equation for the position

Now, consider

$$\dot x = -\mu \nabla V(x) + \sqrt{2 \mu T} \xi$$

The overdamped Langevin equation.

**Exercise:** Plot the equilibrium distribution, in $[0, 2\pi]$

In [ ]:
def euler_x_step(x, f, mu, T, dt):
    return x + f(x)*mu*dt + np.sqrt(2*mu*T*dt)*np.random.normal()

In [ ]:
N_steps = 20000

dt = 0.01
mu = 1
T = 0.4

x = 0

x_data = []
for i in range(N_steps):
    for j in range(10):
        x = euler_x_step(x, lambda x: -np.sin(x), mu, T, dt)
    x_data.append(x)
x_data = np.array(x_data)

In [ ]:
plt.figure()
plt.plot(x_data)

In [ ]:
plt.figure()
count, bins, patches = plt.hist(np.mod(x_data, 2*np.pi), normed=True, bins=32)

# plot the distribution

plt.plot(bins, rho)

## Escape rate

Kramers' theory for the escape rate can be used for Langevin dynamics in a
metastable potential.


In [ ]:
xr = np.linspace(-1.5, 1.5, 101)
plt.figure()

def V_kramers(x):
    return x*(1-x**2)

def f_kramers(x):
    return 3*x**2-1

# V'' = - 6x

plt.plot(xr, V_kramers(xr))
plt.plot(xr, f_kramers(xr))
plt.axhline(0)

In [ ]:
N_steps = 8000

dt = 0.01
mu = 1
T = 1

time_data = []
for realization in range(100):
    x = -np.sqrt(1/3)
    for i in range(N_steps):
        for j in range(10):
            x = euler_x_step(x, f_kramers, mu, T, dt)
        if x>1:
            break
    if i<=N_steps:
        time_data.append(i*10*dt)

In [ ]:
plt.figure()
plt.plot(x_data)

In [ ]:
plt.figure()
count, bins, patches = plt.hist(time_data, normed=True)

fit = [-1, 1]

slope, origin = fit

plt.plot(bins, np.exp(origin+slope*bins))


In [ ]:
delta_E = V_kramers(np.sqrt(1/3)) - V_kramers(-np.sqrt(1/3))

rate = mu/(2*np.pi)*np.exp(-delta_E/T) * np.sqrt(12)


In [ ]:
1/rate